<a href="https://colab.research.google.com/github/siwei-li/LC-graph/blob/master/LCScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import os
os.environ['http_proxy'] = "http://205.134.235.13:3129" 
os.environ['https_proxy'] = "https://205.134.235.13:3129" 

In [8]:
!pip install py2neo -q

In [1]:
import pandas as pd
import time,csv,sys,pytz,os
from pathlib import Path

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select,WebDriverWait

import numpy as np
from datetime import date,datetime,timedelta

In [207]:
# refer following discussion
# https://stackoverflow.com/questions/56829470/selenium-google-colab-error-chromedriver-executable-needs-to-be-in-path
# install chromium, its driver, and selenium
# !apt-get update
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin

# !pip install selenium

from selenium import webdriver
options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [208]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [91]:
# %reload_ext dotenv

In [297]:
from py2neo import Graph, Node, Relationship

g = Graph(f"neo4j+s://{os.getenv('AURA_DOMAIN')}.databases.neo4j.io", auth=("neo4j", os.getenv('AURA_PWD')), routing=True)


In [113]:
print(g.run("MATCH (n) RETURN n"))

 n                                                                          
----------------------------------------------------------------------------
 (_0:Tag {title: 'Array', uri: 'https://leetcode.com/tag/array'})           
 (_1:Tag {title: 'String', uri: 'https://leetcode.com/tag/string'})         
 (_2:Tag {title: 'Hash Table', uri: 'https://leetcode.com/tag/hash-table'}) 



In [40]:
# res = g.run("MATCH (n)-[r]-(n2) RETURN r");
# for r in res:
#     print(r)

In [239]:
driver = webdriver.Chrome('chromedriver', options=options)

In [238]:
driver.quit()

In [240]:
# def login(driver):

url = 'https://leetcode.com/accounts/login'
driver.get(url)
# WebDriverWait(driver, timeout=15).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'sign-in-container__3IFW']").is_displayed())    
btn = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.ID,'signin_btn')))


emailInput = driver.find_element(By.ID, 'id_login')
passwordInput = driver.find_element(By.ID,'id_password')
# loginButton = driver.find_element(By.ID,'signin_btn')
emailInput.send_keys(os.getenv('LC_USERNAME'))
passwordInput.send_keys(os.getenv('LC_PWD'))
# loginButton.click()
driver.execute_script("arguments[0].click();", btn)


#     finally:
#         WebDriverWait(driver, timeout=5).until(lambda x: x.find_element(By.XPATH, "//div[@role = 'table']").is_displayed())

In [36]:
def problemsOnPage(page=1):
    url = 'https://leetcode.com/problemset/all/'
    if page!=1:
      url = f"https://leetcode.com/problemset/all/?page={page}"
    driver.get(url)

    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element(By.XPATH, "//div[@role = 'table']").is_displayed())

    script = '''
const rows = document.querySelectorAll("[role=rowgroup]")[0].children;
const problems = [];
for (const row of rows) {
  const problem = [], cols = row.children;

  const link = cols[1].querySelector('a').href;

  problem.push(link);
  problem.push(cols[1].innerText.trim());
  problem.push(cols[3].innerText.trim());
  problem.push(cols[4].innerText.trim());

  problems.push(problem);
}
return problems
    '''
    problemsData = driver.execute_script(script)
    problems = list(problemsData)
#     problems = pd.DataFrame(problemsData, columns = ['Link','Title','Acceptance','Difficulty'])
    if page==1:
        return problems[1:]
    return problems

In [24]:
g.nodes.match("Problem",uri='https://leetcode.com/problems/two-sum/').first()


Node('Problem', acceptance='49.1%', difficulty='Easy', title='1. Two Sum', uri='https://leetcode.com/problems/two-sum/')

In [37]:
def create_node_from_row(uri, title, acceptance, difficulty):
    if not g.nodes.match("Problem",uri=uri).first():
        n = Node("Problem", uri=uri, title=title, acceptance=acceptance, difficulty=difficulty)
        create_tx.create(n)

In [10]:
from selenium.webdriver.support import expected_conditions as EC

driver.get('https://leetcode.com/problems/length-of-last-word/')
# driver.get('https://leetcode.com/problems/two-sum')

wait = WebDriverWait(driver, 5)
jd = wait.until(EC.presence_of_element_located((By.XPATH, "//div[contains(., 'Similar Questions')]")))
print(jd)

TimeoutException: Message: 
Stacktrace:
#0 0x55a6e1deded3 <unknown>
#1 0x55a6e1bca698 <unknown>
#2 0x55a6e1c06ada <unknown>
#3 0x55a6e1c06cf1 <unknown>
#4 0x55a6e1c42de4 <unknown>
#5 0x55a6e1c2850d <unknown>
#6 0x55a6e1c40940 <unknown>
#7 0x55a6e1c283d3 <unknown>
#8 0x55a6e1bf89ce <unknown>
#9 0x55a6e1bf9d51 <unknown>
#10 0x55a6e1e347c6 <unknown>
#11 0x55a6e1e36d9e <unknown>
#12 0x55a6e1e3759d <unknown>
#13 0x55a6e1e1ddde <unknown>
#14 0x55a6e1e37c8c <unknown>
#15 0x55a6e1e11858 <unknown>
#16 0x55a6e1e53488 <unknown>
#17 0x55a6e1e53628 <unknown>
#18 0x55a6e1e6dbf8 <unknown>
#19 0x7fb229284609 <unknown>


In [284]:
def get_problem_info(url):
    
    driver.get(url)
    WebDriverWait(driver, timeout=20).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'description__24sA']").is_displayed())

    try:
        script = '''
    const l = document.querySelector("div[class = 'description__24sA']").children[5].children[1];
    const elems = l.querySelectorAll('a');
    const tags = [];

    for (elem of elems) {
        tags.push(elem.href);
    }
    return tags
        '''
        tagsData = driver.execute_script(script)
            
    except:
        script = '''
    const l = document.querySelector("div[class = 'description__24sA']").children[4].children[1];
    const elems = l.querySelectorAll('a');
    const tags = [];

    for (elem of elems) {
        tags.push(elem.href);
    }
    return tags
        '''
        tagsData = driver.execute_script(script) 
        return list(tagsData), []
    
    
    try:
        driver.find_element(by=By.XPATH, value="//div[contains(., 'Similar Questions')]")
 
        script = '''
    const l = document.querySelector("div[class = 'description__24sA']").children[6].children[1];
    const elems = l.querySelectorAll('a');
    const tags = [];

    for (elem of elems) {
        tags.push(elem.href);
    }
    return tags
                 '''
        simsData = driver.execute_script(script)
    except:
        simsData = None
    
    
    tags = list(tagsData)
    sims = list(simsData) if simsData else []
    

    return tags, sims

In [350]:
p = g.nodes.match("Problem",uri = 'https://leetcode.com/problems/merge-k-sorted-lists/').first()
p

Node('Problem', acceptance='48.3%', difficulty='Hard', title='23. Merge k Sorted Lists', uri='https://leetcode.com/problems/merge-k-sorted-lists/')

In [352]:
sp = g.nodes.match("Problem",uri = 'https://leetcode.com/problems/valid-parentheses/').first()
sp

Node('Problem', acceptance='40.7%', difficulty='Easy', title='20. Valid Parentheses', uri='https://leetcode.com/problems/valid-parentheses/')

In [355]:
sp = g.nodes.match("Problem",uri = 'https://leetcode.com/problems/merge-two-sorted-lists/').first()
sp

Node('Problem', acceptance='61.8%', difficulty='Easy', title='21. Merge Two Sorted Lists', uri='https://leetcode.com/problems/merge-two-sorted-lists/')

In [356]:
g.match((p, sp), r_type="SIMILAR_TO").all()

[SIMILAR_TO(Node('Problem', acceptance='48.3%', difficulty='Hard', title='23. Merge k Sorted Lists', uri='https://leetcode.com/problems/merge-k-sorted-lists/'), Node('Problem', acceptance='61.8%', difficulty='Easy', title='21. Merge Two Sorted Lists', uri='https://leetcode.com/problems/merge-two-sorted-lists/'))]

In [358]:
g.match((sp, p), r_type="SIMILAR_TO").all()

[SIMILAR_TO(Node('Problem', acceptance='61.8%', difficulty='Easy', title='21. Merge Two Sorted Lists', uri='https://leetcode.com/problems/merge-two-sorted-lists/'), Node('Problem', acceptance='48.3%', difficulty='Hard', title='23. Merge k Sorted Lists', uri='https://leetcode.com/problems/merge-k-sorted-lists/'))]

In [357]:
len(g.match((p, sp), r_type="SIMILAR_TO").all())

1

In [334]:
if len(g.match((p, sp), r_type="SIMILAR_TO").all()) == 0:
    pp = Relationship(p, 'SIMILAR_TO', sp)
    g.create(pp)

In [391]:
def get_problem_info_and_commit(url):
    
    driver.get(url)
    WebDriverWait(driver, timeout=20).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'description__24sA']").is_displayed())
    tagsData, simsData = None, None
    
    try:
        script = '''
    const l = document.querySelector("div[class = 'description__24sA']").children[5].children[1];
    const elems = l.querySelectorAll('a');
    const tags = [];

    for (elem of elems) {
        tags.push(elem.href);
    }
    return tags
        '''
        tagsData = driver.execute_script(script)
        
        try:
            driver.find_element(by=By.XPATH, value="//div[contains(., 'Similar Questions')]")

            script = '''
        const l = document.querySelector("div[class = 'description__24sA']").children[6].children[1];
        const elems = l.querySelectorAll('a');
        const tags = [];

        for (elem of elems) {
            tags.push(elem.href);
        }
        return tags
                     '''
            simsData = driver.execute_script(script)
        except:
            pass
            
    except:
        print(f'{url} only has tags')
        script = '''
    const l = document.querySelector("div[class = 'description__24sA']").children[4].children[1];
    const elems = l.querySelectorAll('a');
    const tags = [];

    for (elem of elems) {
        tags.push(elem.href);
    }
    return tags
        '''
        tagsData = driver.execute_script(script) 
    
    
    
    tags = list(tagsData) if tagsData else []
    sims = list(simsData) if simsData else []
    

    p = g.nodes.match("Problem",uri = url).first()

    try:
        for tag_url in tags:
    #         print(tag_url)
            t = g.nodes.match("Tag",uri=tag_url[:-1]).first()
            print(t)
            if len(g.match((p, t), r_type="BELONGS_TO").all()) == 0:
                print("to be added")
                pt = Relationship(p, 'BELONGS_TO', t)
                g.create(pt)
                print("added")

        for sim_url in sims:
    #             print(sim_url)
            sp = g.nodes.match("Problem",uri=sim_url).first()
            print(sp)
            if len(g.match((p, sp), r_type="SIMILAR_TO").all()) == 0 and len(g.match((sp, p), r_type="SIMILAR_TO").all()) == 0:
                print("to be added")

                pp = Relationship(p, 'SIMILAR_TO', sp)
                g.create(pp)
                print("added")

                
    except Exception as e: 
        print(e)
        print(f"Something went wrong with py2neo processing for {url}")
        new_urls.append(url)

In [339]:
for tag_url in tags:
    
    t = g.nodes.match("Tag",uri=tag_url[:-1]).first()
    print(t)
    if not g.match((p, t), r_type="BELONGS_TO").first():
        print("add")
        pt = Relationship(p, 'BELONGS_TO', t)
        g.create(pt)

(_77:Tag {title: 'Linked List', uri: 'https://leetcode.com/tag/linked-list'})
add
(_81:Tag {title: 'Recursion', uri: 'https://leetcode.com/tag/recursion'})
add


In [ ]:
for sim_url in sims:
#             print(sim_url)
    sp = g.nodes.match("Problem",uri=sim_url).first()
    if not g.match((p, sp), r_type="SIMILAR_TO").first() and not g.match((sp, p), r_type="SIMILAR_TO").first():
        g.merge(SIMILAR_TO(p, sp))

In [348]:
get_problem_info_and_commit('https://leetcode.com/problems/merge-k-sorted-lists/')

### Run

In [367]:
len(url_again)

146

In [364]:
# for page_num in range(1,2):
for page_num in range(2, PAGE_CNT+1):
    print(page_num)

    df = pd.read_csv(f'problem_p{page_num}.csv', usecols=[1,2,3,4])
    probs = df.values.tolist()
    
    for prob in probs[30:]:
        url = prob[0]
        try:
#             tags, sims = get_problem_info(url)
#             find_and_commit(url, tags, sims)
            get_problem_info_and_commit(url)
        except:
            url_again.append(url)
            print(f"Problem {url} encountered problem")
        
    
    print(f"Problems on page {page_num} linked")

2
Problem https://leetcode.com/problems/largest-rectangle-in-histogram/ encountered problem
Problem https://leetcode.com/problems/merge-sorted-array/ encountered problem
Problem https://leetcode.com/problems/gray-code/ encountered problem
Problems on page 2 linked
3
Problem https://leetcode.com/problems/palindrome-partitioning/ encountered problem
Problem https://leetcode.com/problems/binary-tree-preorder-traversal/ encountered problem
Problem https://leetcode.com/problems/binary-tree-postorder-traversal/ encountered problem
Problem https://leetcode.com/problems/lru-cache/ encountered problem
Problem https://leetcode.com/problems/evaluate-reverse-polish-notation/ encountered problem
Problems on page 3 linked
4
Problems on page 4 linked
5
Problem https://leetcode.com/problems/different-ways-to-add-parentheses/ encountered problem
Problems on page 5 linked
6
Problem https://leetcode.com/problems/nim-game/ encountered problem
Problem https://leetcode.com/problems/best-meeting-point/ encou

Problems on page 36 linked
37
Problem https://leetcode.com/problems/frequency-of-the-most-frequent-element/ encountered problem
Problem https://leetcode.com/problems/next-palindrome-using-same-digits/ encountered problem
Problem https://leetcode.com/problems/replace-all-digits-with-characters/ encountered problem
Problem https://leetcode.com/problems/minimum-adjacent-swaps-to-reach-the-kth-smallest-number/ encountered problem
Problems on page 37 linked
38
Problem https://leetcode.com/problems/process-tasks-using-servers/ encountered problem
Problem https://leetcode.com/problems/maximum-number-of-removable-characters/ encountered problem
Problems on page 38 linked
39
Problem https://leetcode.com/problems/painting-a-grid-with-three-different-colors/ encountered problem
Problem https://leetcode.com/problems/check-if-string-is-decomposable-into-value-equal-substrings/ encountered problem
Problem https://leetcode.com/problems/add-minimum-number-of-rungs/ encountered problem
Problem https://

In [365]:
import pickle

with open('caught.pkl', 'wb') as f:
    pickle.dump(url_again, f)

In [365]:
with open('new_caught.pkl', 'wb') as f:
    pickle.dump(new_urls, f)

In [379]:
url_again = new_urls

In [ ]:
new_urls = []

In [392]:
for url in url_again[:5]:
    try:
        get_problem_info_and_commit(url)
    except:
        new_urls.append(url)
        print(f"Problem {url} encountered problem")
        
new_urls

(_1:Tag {title: 'String', uri: 'https://leetcode.com/tag/string'})
to be added
added
(_3:Tag {title: 'Dynamic Programming', uri: 'https://leetcode.com/tag/dynamic-programming'})
to be added
added
(_73:Tag {title: 'Backtracking', uri: 'https://leetcode.com/tag/backtracking'})
to be added
added
(_203:Problem {acceptance: '33.6%', difficulty: 'Hard', title: '132. Palindrome Partitioning II', uri: 'https://leetcode.com/problems/palindrome-partitioning-ii/'})
(_1715:Problem {acceptance: '46.0%', difficulty: 'Hard', title: '1745. Palindrome Partitioning IV', uri: 'https://leetcode.com/problems/palindrome-partitioning-iv/'})
(_2442:Problem {acceptance: '36.4%', difficulty: 'Hard', title: '2472. Maximum Number of Non-overlapping Palindrome Substrings', uri: 'https://leetcode.com/problems/maximum-number-of-non-overlapping-palindrome-substrings/'})
to be added
added
None
to be added
'NoneType' object has no attribute 'labels'
Something went wrong with py2neo processing for https://leetcode.com/p

['https://leetcode.com/problems/intersection-of-two-arrays/',
 'https://leetcode.com/problems/encode-n-ary-tree-to-binary-tree/',
 'https://leetcode.com/problems/design-in-memory-file-system/']

In [377]:
len(new_urls)

43

In [389]:
new_urls

['https://leetcode.com/problems/palindrome-partitioning/',
 'https://leetcode.com/problems/intersection-of-two-arrays/',
 'https://leetcode.com/problems/encode-n-ary-tree-to-binary-tree/',
 'https://leetcode.com/problems/design-in-memory-file-system/',
 'https://leetcode.com/problems/replace-words/',
 'https://leetcode.com/problems/redundant-connection/',
 'https://leetcode.com/problems/find-smallest-letter-greater-than-target/',
 'https://leetcode.com/problems/cheapest-flights-within-k-stops/',
 'https://leetcode.com/problems/spiral-matrix-iii/',
 'https://leetcode.com/problems/rotting-oranges/',
 'https://leetcode.com/problems/available-captures-for-rook/',
 'https://leetcode.com/problems/distance-between-bus-stops/',
 'https://leetcode.com/problems/jump-game-iv/',
 'https://leetcode.com/problems/the-most-recent-three-orders/',
 'https://leetcode.com/problems/the-most-recent-orders-for-each-product/',
 'https://leetcode.com/problems/customer-who-visited-but-did-not-make-any-transacti

### Debug

In [380]:
url = 'https://leetcode.com/problems/largest-rectangle-in-histogram/'

In [381]:
driver.get(url)
WebDriverWait(driver, timeout=20).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'description__24sA']").is_displayed())

True

In [382]:
tagsData, simsData = None, None

try:
    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[5].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
    '''
    tagsData = driver.execute_script(script)

    try:
        driver.find_element(by=By.XPATH, value="//div[contains(., 'Similar Questions')]")

        script = '''
    const l = document.querySelector("div[class = 'description__24sA']").children[6].children[1];
    const elems = l.querySelectorAll('a');
    const tags = [];

    for (elem of elems) {
        tags.push(elem.href);
    }
    return tags
                 '''
        simsData = driver.execute_script(script)
    except:
        pass

except:
    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[4].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
    '''
    tagsData = driver.execute_script(script) 



tags = list(tagsData) if tagsData else []
sims = list(simsData) if simsData else []

In [383]:
tags, sims

(['https://leetcode.com/tag/array/',
  'https://leetcode.com/tag/stack/',
  'https://leetcode.com/tag/monotonic-stack/'],
 ['https://leetcode.com/problems/maximal-rectangle/',
  'https://leetcode.com/problems/maximum-score-of-a-good-subarray/'])

In [386]:
p = g.nodes.match("Problem",uri = url).first()
p

Node('Problem', acceptance='42.2%', difficulty='Hard', title='84. Largest Rectangle in Histogram', uri='https://leetcode.com/problems/largest-rectangle-in-histogram/')

In [387]:
len(g.match((p, t), r_type="BELONGS_TO").all())

0

In [388]:
for tag_url in tags:
    print(tag_url)
    t = g.nodes.match("Tag",uri=tag_url[:-1]).first()
    print(t)
    if len(g.match((p, t), r_type="BELONGS_TO").all()) == 0:
        print("add")
        pt = Relationship(p, 'BELONGS_TO', t)
        g.create(pt)

https://leetcode.com/tag/array/
(_0:Tag {title: 'Array', uri: 'https://leetcode.com/tag/array'})
add
https://leetcode.com/tag/stack/
(_67:Tag {title: 'Stack', uri: 'https://leetcode.com/tag/stack'})
add
https://leetcode.com/tag/monotonic-stack/
(_79:Tag {title: 'Monotonic Stack', uri: 'https://leetcode.com/tag/monotonic-stack'})
add


In [313]:
for tag_url in tags:
#         print(tag_url)
    t = g.nodes.match("Tag",uri=tag_url[:-1]).first()
    print(t)
    if not g.match((p, t), r_type="BELONGS_TO").first():
        pt = Relationship(p, 'BELONGS_TO', t)
        print(pt)
        g.create(pt)

(_77:Tag {title: 'Linked List', uri: 'https://leetcode.com/tag/linked-list'})
(_31)-[:BELONGS_TO {}]->(_77)
(_81:Tag {title: 'Recursion', uri: 'https://leetcode.com/tag/recursion'})
(_31)-[:BELONGS_TO {}]->(_81)


In [347]:
for sim_url in sims:
#             print(sim_url)
    sp = g.nodes.match("Problem",uri=sim_url).first()
    if len(g.match((p, sp), r_type="SIMILAR_TO").all()) == 0:
        pp = Relationship(p, 'SIMILAR_TO', sp)
        g.create(pp)

In [314]:
for sim_url in sims:
#             print(sim_url)
    sp = g.nodes.match("Problem",uri=sim_url).first()
    print(sp)
    if not g.match((p, sp), r_type="SIMILAR_TO").first():
        pp = Relationship(p, 'SIMILAR_TO', sp)
        g.create(pp)

(_33:Problem {acceptance: '48.3%', difficulty: 'Hard', title: '23. Merge k Sorted Lists', uri: 'https://leetcode.com/problems/merge-k-sorted-lists/'})
(_159:Problem {acceptance: '45.8%', difficulty: 'Easy', title: '88. Merge Sorted Array', uri: 'https://leetcode.com/problems/merge-sorted-array/'})
(_219:Problem {acceptance: '54.3%', difficulty: 'Medium', title: '148. Sort List', uri: 'https://leetcode.com/problems/sort-list/'})
(_315:Problem {acceptance: '60.7%', difficulty: 'Medium', title: '244. Shortest Word Distance II', uri: 'https://leetcode.com/problems/shortest-word-distance-ii/'})
(_1604:Problem {acceptance: '53.6%', difficulty: 'Medium', title: '1634. Add Two Polynomials Represented as Linked Lists', uri: 'https://leetcode.com/problems/add-two-polynomials-represented-as-linked-lists/'})
(_1910:Problem {acceptance: '79.3%', difficulty: 'Medium', title: '1940. Longest Common Subsequence Between Sorted Arrays', uri: 'https://leetcode.com/problems/longest-common-subsequence-betwe

In [104]:
get_problem_info('https://leetcode.com/problems/two-sum')

(['https://leetcode.com/tag/array/', 'https://leetcode.com/tag/hash-table/'],
 ['https://leetcode.com/problems/3sum/',
  'https://leetcode.com/problems/4sum/',
  'https://leetcode.com/problems/two-sum-ii-input-array-is-sorted/',
  'https://leetcode.com/problems/two-sum-iii-data-structure-design/',
  'https://leetcode.com/problems/subarray-sum-equals-k/',
  'https://leetcode.com/problems/two-sum-iv-input-is-a-bst/',
  'https://leetcode.com/problems/two-sum-less-than-k/',
  'https://leetcode.com/problems/max-number-of-k-sum-pairs/',
  'https://leetcode.com/problems/count-good-meals/',
  'https://leetcode.com/problems/count-number-of-pairs-with-absolute-difference-k/',
  'https://leetcode.com/problems/number-of-pairs-of-strings-with-concatenation-equal-to-target/',
  'https://leetcode.com/problems/find-all-k-distant-indices-in-an-array/',
  'https://leetcode.com/problems/first-letter-to-appear-twice/',
  'https://leetcode.com/problems/number-of-excellent-pairs/',
  'https://leetcode.com/p

In [56]:
get_problem_info('https://leetcode.com/problems/compare-version-numbers/')

(['https://leetcode.com/tag/two-pointers/',
  'https://leetcode.com/tag/string/'],
 [])

In [30]:
g.nodes.match("Tag",uri='https://leetcode.com/tag/array').first()

Node('Tag', title='Array', uri='https://leetcode.com/tag/array')

In [29]:
p = g.nodes.match("Problem",uri='https://leetcode.com/problems/two-sum/').first()
p

Node('Problem', acceptance='49.1%', difficulty='Easy', title='1. Two Sum', uri='https://leetcode.com/problems/two-sum/')

### Run!

In [228]:
BELONGS_TO = Relationship.type("BELONGS_TO")
SIMILAR_TO = Relationship.type("SIMILAR_TO")

In [269]:
def find_and_commit(p_url, tags, sims):
#     print(p_url, tags, sims)
    p = g.nodes.match("Problem",uri = p_url).first()

    for tag_url in tags:
#         print(tag_url)
        t = g.nodes.match("Tag",uri=tag_url[:-1]).first()
        if not g.match((p, t), r_type="BELONGS_TO").first():
            g.merge(BELONGS_TO(p, t))
    if sims:
        for sim_url in sims:
#             print(sim_url)
            sp = g.nodes.match("Problem",uri=sim_url).first()
            if not g.match((p, sp), r_type="SIMILAR_TO").first() and not g.match((sp, p), r_type="SIMILAR_TO").first():
                g.merge(SIMILAR_TO(p, sp))


In [126]:
PAGE_CNT = 50

In [88]:
for page_num in range(PAGE_CNT+1):

    print(f"Page {page_num} started..")
    probs = problemsOnPage(page=page_num)
    print("Got problem list...")
    
    create_tx = g.begin()
    df = pd.DataFrame(probs)
    df.to_csv(f'problem_p{page_num}.csv')
    df.apply(lambda r: create_node_from_row(r[0],r[1],r[2], r[3]), axis=1)
    create_tx.commit()

    print(f"Page {page_num} problems committed..")
    

Page 3 started..
Got problem list...


/tmp/ipykernel_242839/761596935.py:12: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


Page 3 problems committed..
Page 4 started..
Got problem list...


/tmp/ipykernel_242839/761596935.py:12: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


Page 4 problems committed..
Page 5 started..
Got problem list...


...

/tmp/ipykernel_242839/761596935.py:12: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


Page 47 problems committed..
Page 48 started..
Got problem list...


/tmp/ipykernel_242839/761596935.py:12: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


Page 48 problems committed..
Page 49 started..
Got problem list...
Page 49 problems committed..


/tmp/ipykernel_242839/761596935.py:12: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


In [157]:
url_again = []

In [ ]:
data = {}

In [126]:
PAGE_CNT = 50

In [285]:
for prob in probs[12:20]:
    url = prob[0]
    try:
#             tags, sims = get_problem_info(url)
#             find_and_commit(url, tags, sims)
        get_problem_info_and_commit(url)
    except:
        url_again.append(url)
        print(f"Problem {url} encountered problem")


print(f"Problems on page {page_num} linked")

Problems on page 1 linked


In [230]:
df = pd.read_csv(f'problem_p{page_num}.csv', usecols=[1,2,3,4])
probs = df.values.tolist()

In [232]:
prob = probs[18]
url = prob[0]

In [237]:
try:
    tags, sims = get_problem_info(url)
    print(tags, sims)
except:
    url_again.append(url)
    print(f"Problem {url} encountered problem")

Problem https://leetcode.com/problems/remove-nth-node-from-end-of-list/ encountered problem


In [ ]:
p = g.nodes.match("Problem",uri = p_url).first()

for tag_url in tags:
    print(tag_url)
    t = g.nodes.match("Tag",uri=tag_url[:-1]).first()
    if not g.match((p, t), r_type="BELONGS_TO"):
        g.merge(BELONGS_TO(p, t))
if sims:
    for sim_url in sims:
        print(sim_url)
        sp = g.nodes.match("Problem",uri=sim_url).first()
        if not g.match((p, sp), r_type="SIMILAR_TO") and not g.match((sp, p), r_type="SIMILAR_TO"):
            g.merge(SIMILAR_TO(p, sp))

In [ ]:
find_and_commit(url, tags, sims)
tags, sims = [], []

In [286]:
url = 'https://leetcode.com/problems/roman-to-integer/'

In [287]:
driver.get(url)
WebDriverWait(driver, timeout=20).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'description__24sA']").is_displayed())

True

In [288]:
tagsData, simsData = None, None
    
try:
    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[5].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
    '''
    tagsData = driver.execute_script(script)

    try:
        driver.find_element(by=By.XPATH, value="//div[contains(., 'Similar Questions')]")

        script = '''
    const l = document.querySelector("div[class = 'description__24sA']").children[6].children[1];
    const elems = l.querySelectorAll('a');
    const tags = [];

    for (elem of elems) {
        tags.push(elem.href);
    }
    return tags
                 '''
        simsData = driver.execute_script(script)
    except:
        pass

except:
    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[4].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
    '''
    tagsData = driver.execute_script(script) 



tags = list(tagsData) if tagsData else []
sims = list(simsData) if simsData else []

In [279]:
try:
    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[5].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
    '''
    tagsData = driver.execute_script(script)

except:
    print("exp")
    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[4].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
    '''
    tagsData = driver.execute_script(script) 

In [280]:
tagsData

['https://leetcode.com/tag/hash-table/',
 'https://leetcode.com/tag/math/',
 'https://leetcode.com/tag/string/']

In [281]:
try:
    driver.find_element(by=By.XPATH, value="//div[contains(., 'Similar Questions')]")

    script = '''
const l = document.querySelector("div[class = 'description__24sA']").children[6].children[1];
const elems = l.querySelectorAll('a');
const tags = [];

for (elem of elems) {
    tags.push(elem.href);
}
return tags
             '''
    simsData = driver.execute_script(script)
except:
    simsData = None

In [282]:
tags = list(tagsData)
sims = list(simsData) if simsData else []

In [289]:
sims

['https://leetcode.com/problems/integer-to-roman/']

In [290]:
tags, sims = get_problem_info(url)

In [291]:
tags

['https://leetcode.com/tag/hash-table/',
 'https://leetcode.com/tag/math/',
 'https://leetcode.com/tag/string/']

In [278]:
sims

[]

In [114]:
find_and_commit(url, tags, sims)

In [263]:
p = g.nodes.match("Problem",uri = url).first()
p

Node('Problem', acceptance='52.8%', difficulty='Easy', title='9. Palindrome Number', uri='https://leetcode.com/problems/palindrome-number/')

In [267]:
for tag_url in tags:
    print(tag_url)
    t = g.nodes.match("Tag",uri=tag_url[:-1]).first()
    print(t)
    if not g.match((p, t), r_type="BELONGS_TO").first():
        print("add")
        g.merge(BELONGS_TO(p, t))
        

https://leetcode.com/tag/math/
(_4:Tag {title: 'Math', uri: 'https://leetcode.com/tag/math'})


In [266]:
g.match((p, t), r_type="BELONGS_TO").first()

BELONGS_TO(Node('Problem', acceptance='52.8%', difficulty='Easy', title='9. Palindrome Number', uri='https://leetcode.com/problems/palindrome-number/'), Node('Tag', title='Math', uri='https://leetcode.com/tag/math'))

In [271]:
if sims:
    for sim_url in sims:
        print(sim_url)
        sp = g.nodes.match("Problem",uri=sim_url).first()
        if not g.match((p, sp), r_type="SIMILAR_TO").first() and not g.match((sp, p), r_type="SIMILAR_TO").first():
            g.merge(SIMILAR_TO(p, sp))

https://leetcode.com/problems/palindrome-linked-list/
https://leetcode.com/problems/find-palindrome-with-fixed-length/
https://leetcode.com/problems/strictly-palindromic-number/


In [ ]:
-----------------------------------------------------

In [41]:
create_tx = g.begin()

In [16]:
def get_tags():
    url = 'https://leetcode.com/problemset/all/'
    driver.get(url)

    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element(By.XPATH, "//div[@class = 'relative flex mb-1']").is_displayed())

    script = '''
const data = document.querySelector("div[class='relative flex mb-1']").firstChild.children;
const tags = [];
for (const item of data) {
  const tag = [];

  const link = item.querySelector('a').href;

  tag.push(link);
  tag.push(item.querySelector('span').innerText);

  tags.push(tag);
}
return tags
    '''
    tagsData = driver.execute_script(script)
   
    return list(tagsData)

In [17]:
tags = get_tags()

In [22]:
tgs = tags
tgs

[['https://leetcode.com/tag/array', 'Array'],
 ['https://leetcode.com/tag/string', 'String'],
 ['https://leetcode.com/tag/hash-table', 'Hash Table'],
 ['https://leetcode.com/tag/dynamic-programming', 'Dynamic Programming'],
 ['https://leetcode.com/tag/math', 'Math'],
 ['https://leetcode.com/tag/sorting', 'Sorting'],
 ['https://leetcode.com/tag/greedy', 'Greedy'],
 ['https://leetcode.com/tag/depth-first-search', 'Depth-First Search'],
 ['https://leetcode.com/tag/database', 'Database'],
 ['https://leetcode.com/tag/breadth-first-search', 'Breadth-First Search'],
 ['https://leetcode.com/tag/tree', 'Tree'],
 ['https://leetcode.com/tag/binary-search', 'Binary Search'],
 ['https://leetcode.com/tag/matrix', 'Matrix'],
 ['https://leetcode.com/tag/binary-tree', 'Binary Tree'],
 ['https://leetcode.com/tag/two-pointers', 'Two Pointers'],
 ['https://leetcode.com/tag/bit-manipulation', 'Bit Manipulation'],
 ['https://leetcode.com/tag/stack', 'Stack'],
 ['https://leetcode.com/tag/heap-priority-queue'

In [24]:
tags = pd.DataFrame(tgs)
tags

,0,1
0,https://leetcode.com/tag/array,Array
1,https://leetcode.com/tag/string,String
2,https://leetcode.com/tag/hash-table,Hash Table
3,https://leetcode.com/tag/dynamic-programming,Dynamic Programming
4,https://leetcode.com/tag/math,Math
...,...,...
66,https://leetcode.com/tag/eulerian-circuit,Eulerian Circuit
67,https://leetcode.com/tag/radix-sort,Radix Sort
68,https://leetcode.com/tag/strongly-connected-co...,Strongly Connected Component
69,https://leetcode.com/tag/rejection-sampling,Rejection Sampling


In [51]:
tags.to_csv('tags.csv')

In [50]:
len(tgs)

71

In [27]:
tags.iloc[0,:]

0    https://leetcode.com/tag/array
1                             Array
Name: 0, dtype: object

In [31]:
tgs[0]

['https://leetcode.com/tag/array', 'Array']

In [41]:
create_tx = g.begin()

In [ ]:
n = Node("Tag", uri='https://leetcode.com/tag/array', title='Array')
create_tx.create(n)

In [43]:
g.commit()

/tmp/ipykernel_109057/2607430192.py:1: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


In [46]:
def create_tag_from_row(uri, title):
  n = Node("Tag", uri=uri, title=title)
  create_tx.create(n)

tags.iloc[1:,:].apply(lambda r: create_tag_from_row(r[0],r[1]), axis=1)

g.commit()

TypeError: Cannot run query in closed transaction

In [48]:
create_tx = g.begin()

In [49]:
for uri, title in tgs[1:]:
    n = Node("Tag", uri=uri, title=title)
    create_tx.create(n)

create_tx.commit()

/tmp/ipykernel_109057/2121119833.py:5: DeprecationWarning: The transaction.commit() method is deprecated, use graph.commit(transaction) instead
  create_tx.commit()


In [25]:
driver.quit()



---



In [ ]:
def logTime(startTime, currentTime):
    endTime = time.time()
    currentTime = currentTime or endTime
    hours, rem = divmod(endTime-currentTime, 3600)
    minutes, seconds = divmod(rem, 60)
    print("[Duration:{:0>2}:{:0>2}:{:05.2f}, ".format(int(hours),int(minutes),int(seconds)), end='')
    hours, rem = divmod(endTime-startTime, 3600)
    minutes, seconds = divmod(rem, 60)
    print("Total:{:0>2}:{:0>2}:{:05.2f}] ".format(int(hours),int(minutes),seconds), end='')
    return endTime

In [ ]:
# def export(df, name, path = ''):
#     now = lambda: datetime.now().astimezone().isoformat(timespec='seconds').replace('-','').replace(':','')
#     fullPath = 'Scraped-Data/' + path
#     Path(fullPath).mkdir(parents=True, exist_ok=True)
#     df.to_csv(fullPath + '/' + name + '-' + now() + '.csv')

In [ ]:
def getProblemsByTitle():
    return allProblems.reset_index().set_index('Title')

In [ ]:
def openAndExportAllCompanies():
    url = 'https://leetcode.com/problemset/all/'
    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_tag_name("select").is_displayed())
    driver.find_element_by_css_selector('#expand-company .btn').click()
    script = '''
const buttons = document.getElementById('current-company-tags').children;
const companies = [];
for (const button of buttons) {
  const company = [];
  company.push(button.firstElementChild.innerText.trim());
  company.push(button.firstElementChild.nextElementSibling.innerText.trim());
  company.push(button.href);
  companies.push(company);
}
return companies
    '''
    companiesData = driver.execute_script(script)
    companies = pd.DataFrame(companiesData, columns = ['Name','Count','Link'])
    companies.index = companies.index + 1
    companies.index.name = 'Serial'
    export(companies, 'Companies')
    print("Exported %d Companies (100%%)" % len(companies))
    return companies

In [ ]:
def getFirstOption():
    driver.find_element_by_id('react-select-2--value-item').click()
    firstOption = driver.execute_script("return (4-document.querySelectorAll('#react-select-2--list div[role=option]').length);")
    driver.find_element_by_id('react-select-2--value-item').click()
    return firstOption

In [ ]:
def selectOption(n):
    driver.find_element_by_id('react-select-2--value-item').click()
    driver.find_element_by_id('react-select-2--option-' + str(n)).click()

In [ ]:
def getCompanyProblems():
    script = '''
const rows = document.getElementsByClassName('reactable-data')[0].children;
const problems = [];
for (const row of rows) {
    const problem = [], cols = row.children;
    const tags = Array.from(cols[3].querySelectorAll('a')).map(t => t.innerText.trim()).join(', ');
    const frequency = parseFloat(row.getElementsByClassName('progress-bar')[0].style.width);
    const link = cols[2].querySelector('a').href;
    problem.push(cols[1].innerText.trim());
    problem.push(cols[2].innerText.trim());
    problem.push(tags);
    problem.push(cols[4].innerText.trim());
    problem.push(cols[5].innerText.trim());
    problem.push(frequency);
    problem.push(link);
    problems.push(problem);
}
return problems
    '''
    companyProblemsData = driver.execute_script(script)
    companyProblems = pd.DataFrame(companyProblemsData, columns = ['Id','Title','Tags','Acceptance','Difficulty','Frequency','Link'])
    companyProblems = companyProblems.set_index('Id')
    return companyProblems

In [ ]:
def openAndExportAllCompanyProblems():
    startTime = time.time()
    currentTime = startTime
    optionNames = ['6_months', '1_year', '2_years', 'All_time']
    optionTotalCount = len(optionNames)
    companiesCount = len(allCompanies)
    currentTime = logTime(startTime, currentTime); print("Total Companies: %d" % companiesCount)
    for i in range(companiesCount):
        company = allCompanies.iloc[i]
        companyName = company['Name'].replace(' ','_')
        currentTime = logTime(startTime, currentTime); print("Processing %s (%d/%d) (%d%%)" % (companyName, i+1,companiesCount,((i+1)*100/companiesCount)))
        driver.get(company['Link'])
        WebDriverWait(driver, timeout=30).until(lambda x: x.find_element_by_id('react-select-2--value-item').is_displayed())
        driver.find_element_by_css_selector('input[type=checkbox]').click()
        driver.find_element_by_css_selector('.reactable-column-header th:nth-child(2)').click()
        firstOption = getFirstOption()
        currentTime = logTime(startTime, currentTime); print("Periods available: %d" % (optionTotalCount- firstOption))
        currentTime = logTime(startTime, currentTime); 
        for currentOption in range(firstOption, optionTotalCount):
            optionName = optionNames[currentOption]
            selectOption(currentOption)
            companyProblems = getCompanyProblems()
            print("%s: %d | " % (optionName, len(companyProblems)), end='')
            export(companyProblems, companyName + '-' + optionName, 'Company_Problems/Period-wise/' + optionName)
            export(companyProblems, companyName + '-' + optionName, 'Company_Problems/Name-wise/' + companyName)
        print()
    currentTime = logTime(startTime, currentTime); print("Exported Problems of %d Companies (100%%)" % companiesCount)

In [ ]:
def openAndExportAllInterviews():
    url = 'https://leetcode.com/explore/interview/'
    driver.get(url)
    WebDriverWait(driver, timeout=5).until(lambda x: x.find_element_by_class_name('explore-card').is_displayed())
    script = '''
const cards = Array.from(document.querySelectorAll('.explore-card .premium')).map(e => e.closest('.explore-card'))
const interviews = [];
for (const card of cards) {
  const interview = [];
  interview.push(card.querySelector('.title').innerText.trim());
  interview.push(card.querySelector('.chapter').innerText.trim());
  interview.push(card.querySelector('.item').innerText.trim());
  interview.push(card.querySelector('a').href);
  interviews.push(interview);
}
return interviews;
    '''
    interviewsData = driver.execute_script(script)
    interviews = pd.DataFrame(interviewsData, columns = ['Title','Chapters','Problems','Link'])
    interviews.index = interviews.index + 1
    interviews.index.name = 'Serial'
    export(interviews, 'Interviews')
    print("Exported %d Interviews (100%%)" % len(interviews))
    return interviews

In [ ]:
def joinProblemDetails(problem):
    problem['Id'] = None
    cols = ['Acceptance','Difficulty','Frequency','Link']
    for col in cols:
        problem[col] = None
    try:
        problem['Id'] = allProblemsByTitle.loc[problem['Problem']]['Id']
        for col in cols:
            problem[col] = allProblems.loc[problem['Id']][col]
    except:
        pass
    return problem

In [ ]:
def getInterviewProblems():
    script = '''
const tables = document.getElementsByClassName('table-base');
const problems = [];
for (let i = 1; i <= tables.length; i++) {
  const table = tables[i-1];
  const chapterTitle = table.querySelector('.chapter-title').innerText.trim();
  const items = table.querySelectorAll('*[title="Coding Question"] ~ span');
  for (let j = 1; j <= items.length; j++) {
  const item = items[j-1];
    const itemTitle = item.innerText.trim();
    problems.push([i, chapterTitle, j, itemTitle]);
  }
}
return problems
    '''
    interviewProblemsData = driver.execute_script(script)
    interviewProblems = pd.DataFrame(interviewProblemsData, columns = ['ChapterSerial','Chapter','ProblemSerial','Problem'])
    interviewProblems = interviewProblems.apply(joinProblemDetails, axis=1)
    interviewProblems = interviewProblems.set_index('ChapterSerial')
    return interviewProblems

In [ ]:
def openAndExportAllInterviewProblems():
    startTime = time.time()
    currentTime = startTime
    interviewsCount = len(allInterviews)
    currentTime = logTime(startTime, currentTime); print("Total interviews: %d" % interviewsCount)
    for i in range(interviewsCount):
        interview = allInterviews.iloc[i]
        interviewTitle = interview['Title'].replace(' ','_')
        currentTime = logTime(startTime, currentTime); print("Processing %s (%d/%d) (%d%%)" % (interviewTitle, i+1,interviewsCount,((i+1)*100/interviewsCount)))
        driver.get(interview['Link'])
        WebDriverWait(driver, timeout=10).until(lambda x: str(len(x.find_elements_by_css_selector('.table-base .table-item'))) == interview['Problems'])
        interviewProblems = getInterviewProblems()
        currentTime = logTime(startTime, currentTime); print("Chapters: %s | Problems: %d" % (interview['Chapters'], len(interviewProblems)))
        export(interviewProblems, interviewTitle, 'Interview_Problems')
        currentTime = logTime(startTime, currentTime); print("Exported Problems of %d Interviews (100%%)" % interviewsCount)

In [ ]:
allProblems = openAndExportAllProblems()

Exported 1579 Problems (100%)


In [ ]:
allProblemsByTitle = getProblemsByTitle()

In [ ]:
allCompanies = openAndExportAllCompanies()

Exported 236 Companies (100%)


In [ ]:
openAndExportAllCompanyProblems()

Duration:00:00:01.00, Total:00:06:14.67] Periods available: 2
[Duration:00:00:00.00, Total:00:06:14.67] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:14.81] Processing GE_Digital (138/236) (58%)
[Duration:00:00:02.00, Total:00:06:17.02] Periods available: 2
[Duration:00:00:00.00, Total:00:06:17.02] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:17.13] Processing infosys (139/236) (58%)
[Duration:00:00:01.00, Total:00:06:19.08] Periods available: 1
[Duration:00:00:00.00, Total:00:06:19.08] All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:19.14] Processing Jane_Street (140/236) (59%)
[Duration:00:00:03.00, Total:00:06:22.23] Periods available: 2
[Duration:00:00:00.00, Total:00:06:22.23] 2_years: 1 | All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:22.36] Processing Machine_Zone (141/236) (59%)
[Duration:00:00:01.00, Total:00:06:24.31] Periods available: 1
[Duration:00:00:00.00, Total:00:06:24.31] All_time: 2 | 
[Duration:00:00:00.00, Total:00:06:24.37] 

In [ ]:
allInterviews = openAndExportAllInterviews()

Exported 10 Interviews (100%)


In [ ]:
openAndExportAllInterviewProblems()

[Duration:00:00:00.00, Total:00:00:00.00] Total interviews: 10
[Duration:00:00:00.00, Total:00:00:00.00] Processing Google_Interview (1/10) (10%)
[Duration:00:00:03.00, Total:00:00:03.89] Chapters: 9 | Problems: 81
[Duration:00:00:00.00, Total:00:00:03.89] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:03.89] Processing Facebook (2/10) (20%)
[Duration:00:00:03.00, Total:00:00:07.58] Chapters: 9 | Problems: 79
[Duration:00:00:00.00, Total:00:00:07.58] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:07.58] Processing Microsoft (3/10) (30%)
[Duration:00:00:03.00, Total:00:00:10.62] Chapters: 8 | Problems: 58
[Duration:00:00:00.00, Total:00:00:10.62] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00.00, Total:00:00:10.62] Processing Amazon (4/10) (40%)
[Duration:00:00:03.00, Total:00:00:13.72] Chapters: 8 | Problems: 68
[Duration:00:00:00.00, Total:00:00:13.72] Exported Problems of 10 Interviews (100%)
[Duration:00:00:00

In [ ]:
driver.quit()